In [1]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [250]:
hltb = requests.get('https://howlongtobeat.com/game.php?id=4250').content
hltb_soup = BeautifulSoup(hltb, "lxml")

def get_game_title(soup):
    return soup.find('div', attrs = {'class': 'profile_header shadow_text'}).get_text().strip('\n').strip(' $')

get_game_title(hltb_soup)

'Half-Life 2: Episode One'

In [15]:
def text_from_tag(tag):
    return tag.get_text().strip('\n').strip(' $')

In [251]:
def get_average_time(times):
    return text_from_tag(times[1])

def get_median_time(times):
    return text_from_tag(times[2])

def get_rushed_time(times):
    return text_from_tag(times[3])

def get_leisure_time(times):
    return text_from_tag(times[4])

def get_poll_count(times):
    return text_from_tag(times[0])

In [252]:
def get_main_story_times(soup):
    return soup.find("td", text="Main Story").find_next_siblings("td")

def get_main_story_extras_times(soup):
    return soup.find("td", text="Main + Extras").find_next_siblings("td")

def get_completionist_times(soup):
    return soup.find("td", text="Completionists").find_next_siblings("td")

def get_all_playstyles_times(soup):
    return soup.find("td", text="All PlayStyles").find_next_siblings("td")

In [253]:
def generate_row(tag, soup, play_title):
    return pd.DataFrame(data = [[get_game_title(soup), play_title, get_poll_count(tag), 
                                 get_average_time(tag), get_median_time(tag), get_rushed_time(tag), 
                                 get_leisure_time(tag)]],
                       columns = ['Title', 'Playstyle', 'Polled', 'Average', 'Median', 'Rushed', 'Leisure'])

def cook_soup(id_no):
    hltb = requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content
    hltb_soup = BeautifulSoup(hltb, "lxml")
    return hltb_soup

def cook_soup(id_no):
    return BeautifulSoup(requests.get(str('https://howlongtobeat.com/game.php?id=' + id_no)).content, "lxml")

def create_game_entry(id_no):
    soup = cook_soup(id_no)
    valid_page = [soup.find("td", text="Main Story"), soup.find("td", text="Main + Extras"),
                 soup.find("td", text="Completionists"), soup.find("td", text="All PlayStyles")]
    if None in valid_page:
        return
    else:
        if soup.findAll('tr'):
            main = generate_row(get_main_story_times(soup), soup, "Main")
            main_extra = generate_row(get_main_story_extras_times(soup), soup, "Main + Extras")
            completionist = generate_row(get_completionist_times(soup), soup, "Completionist")
            all_playstyles = generate_row(get_all_playstyles_times(soup), soup, "All Playstyles")
            return main.append(main_extra).append(completionist).append(all_playstyles).set_index(['Title', 'Playstyle'])


half_life_episode_two = create_game_entry("4250")
half_life_episode_two

Polled Average Median  Rushed  Leisure
Title                    Playstyle                                            
Half-Life 2: Episode One Main              791  3h 55m     4h  2h 40m   6h 58m
                         Main + Extras     217  4h 30m     4h  3h 07m   7h 45m
                         Completionist     185  5h 32m     5h  3h 23m  18h 38m
                         All Playstyles   1.2K  4h 16m     4h  2h 46m  18h 04m

In [254]:
rdr2 = create_game_entry('27100')
rdr2

Polled   Average    Median    Rushed  \
Title                 Playstyle                                             
Red Dead Redemption 2 Main              161   43h 34m   43h 57m   31h 06m   
                      Main + Extras     323   74h 15m       70h   51h 02m   
                      Completionist      36  139h 59m  141h 49m  106h 02m   
                      All Playstyles    520   69h 18m       64h   42h 33m   

                                       Leisure  
Title                 Playstyle                 
Red Dead Redemption 2 Main             52h 23m  
                      Main + Extras   124h 03m  
                      Completionist   170h 57m  
                      All Playstyles  133h 49m

In [256]:
foo = np.arange(101, 200)
foo

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199])

In [257]:
bar = []

for i in foo:
    data = create_game_entry(str(i))
    bar.append(data)

pd.concat(bar)

Polled  \
Title                                              Playstyle               
7th Dragon                                         Main                2   
                                                   Main + Extras       2   
                                                   Completionist       1   
                                                   All Playstyles      5   
7th Dragon 2020                                    Main                6   
                                                   Main + Extras       1   
                                                   Completionist       4   
                                                   All Playstyles     11   
99 Levels to Hell                                  Main                1   
                                                   Main + Extras       1   
                                                   Completionist       2   
                                                   All Playstyles      4   
999: Nine Hours, Nine Persons, Nine Doors          Main              140   
                                                   Main + Extras      67   
                                                   Completionist     247   
                                                   All Playstyles    454   
A Boy and His Blob                                 Main               14   
                                                   Main + Extras      11   
                                                   Completionist      19   
                                                   All Playstyles     44   
A Bug's Life                                       Main                7   
                                                   Main + Extras       8   
                                                   Completionist       7   
                                                   All Playstyles     22   
A Game of Dwarves                                  Main                1   
                                                   Main + Extras       2   
                                                   Completionist       2   
                                                   All Playstyles      5   
A Hat in Time                                      Main               99   
                                                   Main + Extras     114   
...                                                                  ...   
Ace Combat X: Skies of Deception                   Completionist       2   
                                                   All Playstyles     19   
Ace Combat: Assault Horizon - Enhanced Edition     Main               41   
                                                   Main + Extras       2   
                                                   Completionist       5   
                                                   All Playstyles     48   
Ace Combat: Assault Horizon Legacy                 Main               21   
                                                   Main + Extras       9   
                                                   Completionist       2   
                                                   All Playstyles     32   
Achievement Unlocked                               Main                5   
                                                   Main + Extras       4   
                                                   Completionist      10   
                                                   All Playstyles     19   
ActRaiser                                          Main               33   
                                                   Main + Extras      21   
                                                   Completionist      15   
                                                   All Playstyles     69   
Actual Sunlight                                    Main               46   
                                                   Main + Extras       7   
                                      